# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("Output_Data_File/cities.csv")
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hambantota,6.12,81.12,77.00,100,40,4.70,LK,1603331810
1,Codrington,-38.27,141.97,61.21,68,65,6.76,AU,1603331559
2,Ushuaia,-54.80,-68.30,50.00,53,75,10.29,AR,1603331810
3,Cabedelo,-6.98,-34.83,77.00,94,75,5.82,BR,1603331810
4,Yar-Sale,66.83,70.83,19.33,97,91,12.86,RU,1603331811
...,...,...,...,...,...,...,...,...,...
559,Yokadouma,3.52,15.05,67.84,95,100,0.94,CM,1603332365
560,Khurai,24.04,78.33,74.79,35,0,1.30,IN,1603332305
561,Mwingi,-0.93,38.06,64.83,80,90,3.76,KE,1603332366
562,Manjacaze,-24.71,33.88,66.45,96,31,5.10,MZ,1603332366


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]

In [4]:
fig = gmaps.figure(center=(45.0, -5.0), zoom_level=2)
heat_lay = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_lay)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
city_df=weather_df[(weather_df["Max Temp"]>=80)&(weather_df["Max Temp"]<90)&(weather_df["Humidity"]<50)& (weather_df["Lat"]>20)]
city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,Atar,20.52,-13.05,84.09,29,0,2.57,MR,1603331836
96,Laguna,38.42,-121.42,80.60,28,1,1.23,US,1603331910
200,Adrar,20.50,-10.07,83.25,19,3,4.85,MR,1603331997
487,Tessalit,20.20,1.01,80.78,17,0,7.47,ML,1603332288


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df= city_df.loc[:,["City","Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Names"]= ""
hotel_df

,City,Country,Lat,Lng,Hotel Names
29,Atar,MR,20.52,-13.05,
96,Laguna,US,38.42,-121.42,
200,Adrar,MR,20.50,-10.07,
487,Tessalit,ML,20.20,1.01,


In [14]:
# Url and Params for top Hotel

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params={"radius": 5000,
        "keyword": "hotel",
        "type": "hotel",
        "key": g_key}

#Using .itterrow()

for index, row in hotel_df.iterrows():
    
    #get city name in each row
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    #add a keyword to my params dictionary
    params["location"]=f"{lat}, {lng}"

    #Make a query url and API call
    print(f"Retrieving Results for Index {index}: {city_name}.")
    query_url=requests.get(base_url, params=params).json()
    
    #extract results
    results=query_url['results']
    
    #Add the information into the Hotel Names column
    try:
        hotel_df.loc[index, "Hotel Names"]=results[0]['name']
        print(f"Closest hotel in (city_name) is {results[0]['name']}.")
    
    except (KeyError, IndexError):
        print("There is a missing field. Skipping...")
    print("----------")
    
    #tme sleep for API
    time.sleep(1)
    
# Ending Print (end of search)
print("-----End of Search------")

Retrieving Results for Index 29: Atar.
Closest hotel in (city_name) is Auberge mer et desert.
----------
Retrieving Results for Index 96: Laguna.
Closest hotel in (city_name) is Hilton Garden Inn Sacramento Elk Grove.
----------
Retrieving Results for Index 200: Adrar.
There is a missing field. Skipping...
----------
Retrieving Results for Index 487: Tessalit.
There is a missing field. Skipping...
----------
-----End of Search------


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Names}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))